#### Modifying Linear Search (WASM)

In [1]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [2]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [3]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

---

Compile the P0 program below by running the next two cells:

In [4]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

In [5]:
compileString("""
const N = 5
var a: [1 .. N] → integer
program linearsearch
    var i, x: integer
        x ← read(); i := 0 // read number to be searched
        while i < N do i := i + 1; a[i] ← read()  // read array elements
        while (i > 0) and (a[i] ≠ x) do i := i - 1
        writeln(); write(i) // write index of last occurrence of x or 0 if not found
""", 'linearsearch.wat')

 You can convert `linearsearch.wat` to a binary file and run it using one of the three methods above for running WebAssembly in Jupyter:

In [6]:
!wat2wasm linearsearch.wat

In [7]:
runwasm("linearsearch.wasm")

<IPython.core.display.Javascript object>

In [ ]:
runpywasm("linearsearch.wasm")

In [ ]:
runwasmer("linearsearch.wasm")

Now inspect the generated code by running the cell below or leaving out the last parameter of `compileString` and using `print` to print the resulting string. Explain the role of each line!

In [8]:
!cat linearsearch.wat

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 20)
(func $program
(local $i i32)
(local $x i32)
(local $0 i32)
call $read
local.set $x
i32.const 0
local.set $i
loop
local.get $i
i32.const 5
i32.lt_s
if
local.get $i
i32.const 1
i32.add
local.set $i
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
call $read
i32.store
br 1
end
end
loop
local.get $i
i32.const 0
i32.gt_s
if (result i32)
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load
local.get $x
i32.ne
else
i32.const 0
end
if
local.get $i
i32.const 1
i32.sub
local.set $i
br 1
end
end
call $writeln
local.get $i
call $write
)
(memory 1)
(start $program)
)

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 20)
(func $program
(local $i i32)
(local $x i32)
(local $0 i32)

; x ← read()
call $read        
local.set $x

i := 0
i32.const 0
local.set $i


loop

; checks i < 5
local.get $i
i32.const 5
i32.lt_s

if

i := i + 1
local.get $i
i32.const 1
i32.add
local.set $i

; (i - 1) * 4 + 0, calculates the memeory address given index i - a[i]
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add

; reads the input, and stores the input value to the memory address calculated previously
call $read
i32.store

; jumps to the second closest enclosing block
br 1
end
end

loop

; i > 0
local.get $i
i32.const 0
i32.gt_s
if (result i32)

; (i - 1) * 4 + 0, calculates the memory address given index i - a[i]
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add

; load the value from the memory address calculated previously
; and compare it with x
i32.load
local.get $x
i32.ne
else
i32.const 0 ;; 
end

if
local.get $i
i32.const 1
i32.sub
local.set $i ; i := i - 1
br 1
end
end
call $writeln ; writeln()
local.get $i
call $write   ; write(i)
)
(memory 1)    ; set up memory
(start $program)  ; call program
)

Perform the following modifications of the above program; it is repeated below:

1. Make variables `i` and `x` global rather than local to the main program. What changes do you observe in the generated code?
2. Make array `a` a local variable of the main program. What changes do you observe in the generated code?
3. Make constant `N` a local constant of the main program. What changes do you observe in the generated code?

In [15]:
wasm = compileString("""
var i, x: integer
program linearsearch
    const N = 5
    var a: [1 .. N] → integer
        x ← read(); i := 0 // read number to be searched
        while i < N do i := i + 1; a[i] ← read()  // read array elements
        while (i > 0) and (a[i] ≠ x) do i := i - 1
        writeln(); write(i) // write index of last occurrence of x or 0 if not found
""")
print(wasm)

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $i (mut i32) i32.const 0)
(global $x (mut i32) i32.const 0)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $a i32)
(local $0 i32)
(local $_fp i32)
global.get $_memsize
local.set $_fp
global.get $_memsize
i32.const 20
i32.add
local.tee $a
global.set $_memsize
call $read
global.set $x
i32.const 0
global.set $i
loop
global.get $i
i32.const 5
i32.lt_s
if
global.get $i
i32.const 1
i32.add
global.set $i
global.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
local.get $a
i32.add
call $read
i32.store
br 1
end
end
loop
global.get $i
i32.const 0
i32.gt_s
if (result i32)
global.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
local.get $a
i32.add
i32.load
global.get $x
i32.ne
else
i32.const 0
end
if
global.get $i
i32.const 1
i32.sub
global.set $i
br 1
end
end
call $writeln
global.get $i
call $write
local.get $_fp
global.set $_

Your answer here.